In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## CIFAR-10 case
- cifar-10 데이터로 MLP, CNN 차이 확인
- CNN layer 직접 설계해보고, 전이학습(transfer learning, EfficientNetB0)도 해보기
- Colab, LMS session 등의 문제로 kaggle-notebook 통해서 실험

### 01 MLP실험
- MLP가 cifar-10같은 이미지 데이터에 한계 있음을 인지

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10

(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

# 훈련 데이터셋에서 20%를 검증 데이터셋으로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, 
                                                  test_size=0.2, 
                                                  random_state=42)
                                                  
# 훈련, 검증, 테스트 데이터와 레이블 종류가 몇개인지 출력합니다.
print("전체 학습 데이터: {} 레이블: {}".format(x_train_full.shape, y_train_full.shape))
print("학습 데이터: {} 레이블: {}".format(x_train.shape, y_train.shape))
print("검증 데이터: {} 레이블: {}".format(x_val.shape, y_val.shape))
print("테스트 데이터: {} 레이블: {}".format(x_test.shape, y_test.shape))

# cifar10의 분류에 해당하는 'airplane', 'automobile', 'bird', 'cat', 'deer', 
# 'dog', 'frog', 'horse', 'ship', 'truck'를 class_name으로 정의합니다.
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

x_train = x_train.reshape(-1, 32*32*3)
x_val = x_val.reshape(-1, 32*32*3)
x_test = x_test.reshape(-1, 32*32*3)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

# 훈련, 검증, 테스트 데이터를 255로 나누어 0~1 사이의 값으로 변환합니다.
x_train = x_train / 255.
x_val = x_val / 255.
x_test = x_test / 255.

2025-06-11 17:21:07.166780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749662467.543545      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749662467.655157      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
전체 학습 데이터: (50000, 32, 32, 3) 레이블: (50000, 1)
학습 데이터: (40000, 32, 32, 3) 레이블: (40000, 1)
검증 데이터: (10000, 32, 32, 3) 레이블: (10000, 1)
테스트 데이터: (10000, 32, 32, 3) 레이블: (10000, 1)
(40000, 3072)
(10000, 3072)
(10000, 3072)


In [2]:
from tensorflow.keras import models, layers

# 모델 구성 시작
model = models.Sequential()
# 입력층 (flatten된 이미지 데이터)
model.add(layers.Input(shape=(3072,)))

# 첫 번째 은닉층
model.add(layers.Dense(2048, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

# 두 번째 은닉층
model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

# 세 번째 은닉층
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))

# 출력층 (CIFAR-10은 10개의 클래스)
model.add(layers.Dense(10, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', # rmsprop에서 adam으로 변경
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

I0000 00:00:1749662511.519848      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749662511.520495      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 콜백 정의(ModelCheckPoint, EarlyStopping)
check_point_cb = ModelCheckpoint(
    filepath = 'cifar10_model.keras',
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    verbose = 1
)

early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,                          # 개선 없으면 중단
    restore_best_weights=True,           # 가장 좋은 가중치 복원
    verbose=1
)

history_pj3_3 = model.fit(x_train, y_train,
                          epochs = 60,
                          validation_data = (x_val, y_val),
                          callbacks=[check_point_cb, early_stopping_cb])

Epoch 1/60


I0000 00:00:1749662527.202216     100 service.cc:148] XLA service 0x131e5150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749662527.203739     100 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749662527.203760     100 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749662527.676810     100 cuda_dnn.cc:529] Loaded cuDNN version 90300


  36/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.1237 - loss: 3.4868    

I0000 00:00:1749662530.544579     100 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1249/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2343 - loss: 2.5067
Epoch 1: val_loss improved from inf to 1.79651, saving model to cifar10_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.2343 - loss: 2.5062 - val_accuracy: 0.3595 - val_loss: 1.7965
Epoch 2/60
1237/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3426 - loss: 1.8446
Epoch 2: val_loss did not improve from 1.79651
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.3426 - loss: 1.8445 - val_accuracy: 0.3232 - val_loss: 1.9378
Epoch 3/60
1246/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3607 - loss: 1.7953
Epoch 3: val_loss did not improve from 1.79651
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.3607 - loss: 1.7952 - val_accuracy: 0.3507 - val_loss: 1.8012
Epoch 4/60
1239/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3713 - loss: 1.7529
Epoch 4: val_loss improved from 1.79651 to 1.71995, saving model to cifar10_model.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 6s 5

In [4]:
# 테스트 데이터셋을 이용해 모델을 평가합니다.
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4807 - loss: 1.4466


[1.4536702632904053, 0.4771000146865845]

### 02 CNN실험
- CNN 실험 내용
  - 초기 : 간단한 CNN 구조 적용
  - 실험-1 : 데이터 증강, 모델층 깊게 하기
  - 실험-2 : 학습 스케쥴러, 분류층 깊게 만들기
  - 전이학습 1차 EfficientNetB0(초기 학습 epoch = 50, fine-tuning epoch = 30)
  - 전이학습 2차 EfficientNetB0(초기 학습 epoch = 60, fine-tuning epoch = 60)
    - 거의 2시간 가까이 소모

### 02-01 CNN 초기
- 간단한 형태의 CNN 구조 적용

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10

(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

# 훈련 데이터셋에서 20%를 검증 데이터셋으로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full,
                                                  test_size=0.2,
                                                  random_state=42)

# 정규화
x_train = x_train.astype('float32')/255.0
x_val = x_val.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

2025-06-13 08:05:00.715728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749801900.965589      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749801901.037564      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [8]:
from tensorflow.keras import models, layers

# --- CNN 모델 정의 ---
model = models.Sequential()

# 입력 형태는 (32, 32, 3)입니다. CIFAR-10 등 RGB 이미지
model.add(layers.Input(shape=(32, 32, 3)))

# 특징 추출 부분 (Feature Extraction)
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))  # 대소문자 정확히

model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# 분류 부분 (Classifier)
model.add(layers.Flatten())  # 2D → 1D
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))  # 10-class 분류

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

I0000 00:00:1749804312.849949      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749804312.850735      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 콜백 정의(ModelCheckPoint, EarlyStopping)
check_point_cb = ModelCheckpoint(
    filepath = 'cifar10_model_cnn.keras',
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    verbose = 1
)

early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,                          # 개선 없으면 중단
    restore_best_weights=True,           # 가장 좋은 가중치 복원
    verbose=1
)

history_cnn_1 = model.fit(x_train, y_train,
                          epochs = 60,
                          validation_data = (x_val, y_val),
                          callbacks=[check_point_cb, early_stopping_cb])

Epoch 1/60


I0000 00:00:1749804342.822870     111 service.cc:148] XLA service 0x7f575800aba0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749804342.823452     111 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1749804342.823469     111 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1749804343.127423     111 cuda_dnn.cc:529] Loaded cuDNN version 90300


  57/1250 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.1165 - loss: 2.3086

I0000 00:00:1749804346.049189     111 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1238/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3024 - loss: 1.8747
Epoch 1: val_loss improved from inf to 1.32480, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.3033 - loss: 1.8726 - val_accuracy: 0.5271 - val_loss: 1.3248
Epoch 2/60
1249/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4941 - loss: 1.3974
Epoch 2: val_loss improved from 1.32480 to 1.17087, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.4941 - loss: 1.3973 - val_accuracy: 0.5947 - val_loss: 1.1709
Epoch 3/60
1243/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5570 - loss: 1.2407
Epoch 3: val_loss improved from 1.17087 to 1.06114, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5570 - loss: 1.2406 - val_accuracy: 0.6280 - val_loss: 1.0611
Epoch 4/60
1245/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5860 - loss: 1.1495
Epoch 4: val_loss improved fr

In [10]:
# 테스트 데이터셋을 이용해 모델을 평가합니다.
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7020 - loss: 0.8889


[0.8940004706382751, 0.704200029373169]

#### 02-02 CNN 실험-1
- 데이터 증강
- 모델층을 더 깊게하기

In [5]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras.datasets import cifar10

(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

# 훈련 데이터셋에서 20%를 검증 데이터셋으로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, 
                                                  test_size=0.2, 
                                                  random_state=42)

# 정규화
x_train = x_train.astype('float32')/255.0
x_val = x_val.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

In [8]:
# 데이터 증강
data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])

# --- CNN 모델 정의 ---
model = models.Sequential()

# 입력 형태는 (32, 32, 3) 입니다. # 이미지를 그대로 받음
model.add(layers.Input(shape=(32, 32, 3)))
model.add(data_augmentation) # <-- 데이터 증강 레이어를 모델의 맨 앞에 추가

# 특징 추출 부분 (Feature Extraction)

model.add(layers.Conv2D(64, (3, 3), padding='same'))
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu'))   # <-- 활성화 함수를 분리
model.add(layers.Conv2D(64, (3, 3), padding='same'))
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu'))   # <-- 활성화 함수를 분리
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), padding='same')) #<-- 필터 수 증가
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu')) # <-- 활성화 함수를 분리
model.add(layers.Conv2D(128, (3, 3), padding='same')) #<-- Conv 레이어 추가
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu')) # <-- 활성화 함수를 분리
model.add(layers.MaxPooling2D((2, 2)))

# 분류 부분 (Classifier)
model.add(layers.Flatten()) # 2D 특징 맵을 1D 벡터로 변환
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax')) # 10개 클래스로 분류

# 모델 컴파일
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 콜백 정의(ModelCheckPoint, EarlyStopping)
check_point_cb = ModelCheckpoint(
    filepath = 'cifar10_model_cnn.keras',
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    verbose = 1
)

early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,                          # 개선 없으면 중단
    restore_best_weights=True,           # 가장 좋은 가중치 복원
    verbose=1
)

history_cnn_2 = model.fit(x_train, y_train,
                          epochs = 60,
                          validation_data = (x_val, y_val),
                          callbacks=[check_point_cb, early_stopping_cb])

Epoch 1/60
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1506 - loss: 2.3472
Epoch 1: val_loss improved from inf to 2.00367, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 30s 16ms/step - accuracy: 0.1506 - loss: 2.3468 - val_accuracy: 0.1966 - val_loss: 2.0037
Epoch 2/60
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1436 - loss: 2.1310
Epoch 2: val_loss improved from 2.00367 to 1.95110, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1437 - loss: 2.1310 - val_accuracy: 0.2015 - val_loss: 1.9511
Epoch 3/60
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1475 - loss: 2.1026
Epoch 3: val_loss did not improve from 1.95110
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - accuracy: 0.1475 - loss: 2.1026 - val_accuracy: 0.2250 - val_loss: 1.9540
Epoch 4/60
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.1574 - loss: 2.0906
Epoch 4: val_loss improved from 1.95110 to 1.89289, sa

In [10]:
# 테스트 데이터셋을 이용해 모델을 평가합니다.
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7405 - loss: 0.7974


[0.8025781512260437, 0.739300012588501]

#### 02-03 CNN-실험2
- 좀 더 fine-tuning할게 있는지?

In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers, optimizers  # optimizers추가
from tensorflow.keras.datasets import cifar10

(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

# 훈련 데이터셋에서 20%를 검증 데이터셋으로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, 
                                                  test_size=0.2, 
                                                  random_state=42)

# 정규화
x_train = x_train.astype('float32')/255.0
x_val = x_val.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

2025-06-11 18:36:46.044844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749667006.477768      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749667006.595413      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [2]:
# 데이터 증강
data_augmentation = models.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])

# --- CNN 모델 정의 ---
model = models.Sequential()

# 입력 형태는 (32, 32, 3) 입니다. # 이미지를 그대로 받음
model.add(layers.Input(shape=(32, 32, 3)))
model.add(data_augmentation) # <-- 데이터 증강 레이어를 모델의 맨 앞에 추가

# 특징 추출 부분 (Feature Extraction)

model.add(layers.Conv2D(64, (3, 3), padding='same'))
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu'))   # <-- 활성화 함수를 분리
model.add(layers.Conv2D(64, (3, 3), padding='same'))
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu'))   # <-- 활성화 함수를 분리
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3, 3), padding='same')) #<-- 필터 수 증가
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu')) # <-- 활성화 함수를 분리
model.add(layers.Conv2D(128, (3, 3), padding='same')) #<-- Conv 레이어 추가
model.add(layers.BatchNormalization()) # <-- 배치 정규화 추가
model.add(layers.Activation('relu')) # <-- 활성화 함수를 분리
model.add(layers.MaxPooling2D((2, 2)))

# 분류 부분 (Classifier)
model.add(layers.Flatten()) # 2D 특징 맵을 1D 벡터로 변환
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(512, activation='relu')) # <-- 유닛 수를 늘리고
model.add(layers.BatchNormalization())          # <-- BN 추가
model.add(layers.Dense(256, activation='relu')) # <-- 층을 하나 더 추가
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax')) # 10개 클래스로 분류

# 모델 컴파일
model.compile(optimizer=optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

I0000 00:00:1749667041.326655      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749667041.327503      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [3]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
# ReduceLROnPlateau 추가! callback 설정 추가목적
import time

st = time.time()

# 콜백 정의(ModelCheckPoint, EarlyStopping)
check_point_cb = ModelCheckpoint(
    filepath = 'cifar10_model_cnn.keras',
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    verbose = 1
)

early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    patience=10,                          # 개선 없으면 중단
    restore_best_weights=True,           # 가장 좋은 가중치 복원
    verbose=1
)

reduce_lr_cb = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    patience = 3, # 3 epoch 동안 개선 없으면 실행
    verbose = 1
)

history_cnn_3 = model.fit(x_train, y_train,
                          epochs = 60,
                          validation_data = (x_val, y_val),
                          callbacks=[check_point_cb, early_stopping_cb, reduce_lr_cb])

ed = time.time()

print(f'소모시간: {round(ed-st,2)}초')

Epoch 1/60


I0000 00:00:1749667094.652926      97 cuda_dnn.cc:529] Loaded cuDNN version 90300


1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3517 - loss: 1.8550
Epoch 1: val_loss improved from inf to 1.81224, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 33s 17ms/step - accuracy: 0.3518 - loss: 1.8544 - val_accuracy: 0.4026 - val_loss: 1.8122 - learning_rate: 0.0010
Epoch 2/60
1247/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5319 - loss: 1.3242
Epoch 2: val_loss improved from 1.81224 to 1.24675, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.5320 - loss: 1.3240 - val_accuracy: 0.5734 - val_loss: 1.2468 - learning_rate: 0.0010
Epoch 3/60
1247/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6057 - loss: 1.1405
Epoch 3: val_loss improved from 1.24675 to 1.03989, saving model to cifar10_model_cnn.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.6057 - loss: 1.1404 - val_accuracy: 0.6438 - val_loss: 1.0399 - learning_rate: 0.0010
Epoch 4/60
1249/1250 ━━━━━━━━━━━━━━━━

In [4]:
# 테스트 데이터셋을 이용해 모델을 평가합니다.
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8054 - loss: 0.5809


[0.5742959380149841, 0.807200014591217]

#### 02-04 전이학습-1
- 1차 학습 epoch 50, 2차 학습 epoch 30

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
st = time.time()

# 1. 데이터 로드 및 분할
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

# 2. 데이터 증강 레이어 정의
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="data_augmentation")

# --- 모델 구성 ---
# 3. 기본 모델 로드
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(75, 75, 3))
base_model.trainable = False  # 동결

# 4. Functional API를 사용한 전체 모델 조립
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = layers.Resizing(75, 75)(x)
x = preprocess_input(x) # 수동 정규화 대신 여기서 처리
x = base_model(x, training=False) # training=False 설정이 중요!
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs, outputs)

# --- 1차 학습 ---
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks_list = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint(filepath='cifar10_transfer_best.keras', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
]

history = model.fit(x_train, y_train,
                    epochs=50, # 에포크는 조절 가능
                    validation_data=(x_val, y_val),
                    callbacks=callbacks_list)

# --- 2차 학습 (미세 조정) ---
base_model.trainable = True # 전체 동결 해제

# (선택적) 일부만 동결 해제
# for layer in base_model.layers[:-20]:
#     layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), # 매우 낮은 학습률
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# initial_epoch를 설정하여 학습 이어가기
fine_tune_epochs = 30
total_epochs = len(history.epoch) + fine_tune_epochs

fine_tune_history = model.fit(x_train, y_train,
                              epochs=total_epochs,
                              initial_epoch=len(history.epoch), # 이전 학습이 끝난 시점부터 시작
                              validation_data=(x_val, y_val),
                              callbacks=callbacks_list)


ed = time.time()
print(f'소요시간:{round(ed-st,2)}초')

2025-06-11 19:49:54.279552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749671394.505587      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749671394.567141      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


I0000 00:00:1749671414.440988      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749671414.441731      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50


E0000 00:00:1749671433.246681      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1749671435.571496      99 cuda_dnn.cc:529] Loaded cuDNN version 90300


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5376 - loss: 1.3295
Epoch 1: val_loss improved from inf to 0.64236, saving model to cifar10_transfer_best.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 47s 23ms/step - accuracy: 0.5377 - loss: 1.3294 - val_accuracy: 0.7807 - val_loss: 0.6424 - learning_rate: 0.0010
Epoch 2/50
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6511 - loss: 0.9919
Epoch 2: val_loss improved from 0.64236 to 0.58537, saving model to cifar10_transfer_best.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.6511 - loss: 0.9919 - val_accuracy: 0.8017 - val_loss: 0.5854 - learning_rate: 0.0010
Epoch 3/50
1247/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6683 - loss: 0.9541
Epoch 3: val_loss did not improve from 0.58537
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.6683 - loss: 0.9541 - val_accuracy: 0.7955 - val_loss: 0.5916 - learning_rate: 0.0010
Epoch 4/50
1249/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6795 

E0000 00:00:1749672421.036627      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.4237 - loss: 2.9561
Epoch 42: val_loss did not improve from 0.49551
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 156s 79ms/step - accuracy: 0.4237 - loss: 2.9557 - val_accuracy: 0.6437 - val_loss: 1.3658 - learning_rate: 1.0000e-05
Epoch 43/71
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.5563 - loss: 1.7903
Epoch 43: val_loss did not improve from 0.49551
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 93s 75ms/step - accuracy: 0.5564 - loss: 1.7902 - val_accuracy: 0.7001 - val_loss: 1.0660 - learning_rate: 1.0000e-05
Epoch 44/71
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6094 - loss: 1.4003
Epoch 44: val_loss did not improve from 0.49551
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 93s 75ms/step - accuracy: 0.6094 - loss: 1.4003 - val_accuracy: 0.7345 - val_loss: 0.8703 - learning_rate: 1.0000e-05
Epoch 45/71
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.6397 - loss: 1.2004
Epoch 45: val_loss did not improve from 0.49551
1250/1250 ━━

In [2]:
# 테스트 데이터셋을 이용해 모델을 평가합니다.
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9055 - loss: 0.2709


[0.27596190571784973, 0.9049000144004822]

#### 02-05 전이학습_v2
- 전이학습시 충분히 학습이 된 것 같지 않아서 epoch수만 변경
  - 1차 학습 epoch 60(기존 50), 2차 학습 epoch 60(기존 30)

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
st = time.time()

# 1. 데이터 로드 및 분할
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

# 2. 데이터 증강 레이어 정의
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="data_augmentation")

# --- 모델 구성 ---
# 3. 기본 모델 로드
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(75, 75, 3))
base_model.trainable = False  # 동결

# 4. Functional API를 사용한 전체 모델 조립
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = layers.Resizing(75, 75)(x)
x = preprocess_input(x) # 수동 정규화 대신 여기서 처리
x = base_model(x, training=False) # training=False 설정이 중요!
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs, outputs)

# --- 1차 학습 ---
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

callbacks_list = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint(filepath='cifar10_transfer_best.keras', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1)
]

history = model.fit(x_train, y_train,
                    epochs=60, # 에포크는 조절 가능
                    validation_data=(x_val, y_val),
                    callbacks=callbacks_list)

# --- 2차 학습 (미세 조정) ---
base_model.trainable = True # 전체 동결 해제

# (선택적) 일부만 동결 해제
# for layer in base_model.layers[:-20]:
#     layer.trainable = False

model.compile(optimizer=optimizers.Adam(learning_rate=1e-5), # 매우 낮은 학습률
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# initial_epoch를 설정하여 학습 이어가기
fine_tune_epochs = 60
total_epochs = len(history.epoch) + fine_tune_epochs

fine_tune_history = model.fit(x_train, y_train,
                              epochs=total_epochs,
                              initial_epoch=len(history.epoch), # 이전 학습이 끝난 시점부터 시작
                              validation_data=(x_val, y_val),
                              callbacks=callbacks_list)


ed = time.time()
print(f'소요시간:{round(ed-st,2)}초')

2025-06-13 05:27:48.330757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749792468.522674      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749792468.580613      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


I0000 00:00:1749792486.582303      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1749792486.583070      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/60


E0000 00:00:1749792503.998657      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1749792506.185838     101 cuda_dnn.cc:529] Loaded cuDNN version 90300


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5415 - loss: 1.3129
Epoch 1: val_loss improved from inf to 0.62695, saving model to cifar10_transfer_best.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - accuracy: 0.5415 - loss: 1.3128 - val_accuracy: 0.7862 - val_loss: 0.6270 - learning_rate: 0.0010
Epoch 2/60
1247/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6533 - loss: 1.0036
Epoch 2: val_loss improved from 0.62695 to 0.59109, saving model to cifar10_transfer_best.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.6533 - loss: 1.0035 - val_accuracy: 0.7970 - val_loss: 0.5911 - learning_rate: 0.0010
Epoch 3/60
1248/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6765 - loss: 0.9354
Epoch 3: val_loss improved from 0.59109 to 0.57201, saving model to cifar10_transfer_best.keras
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 23s 18ms/step - accuracy: 0.6765 - loss: 0.9354 - val_accuracy: 0.8000 - val_loss: 0.5720 - learning_rate: 0.0010
Epoch 4/60
1250/1250 ━━━━

E0000 00:00:1749793650.071594      35 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4306 - loss: 2.9134
Epoch 50: val_loss did not improve from 0.50281
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 146s 74ms/step - accuracy: 0.4307 - loss: 2.9131 - val_accuracy: 0.6540 - val_loss: 1.3327 - learning_rate: 1.0000e-05
Epoch 51/109
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.5594 - loss: 1.7556
Epoch 51: val_loss did not improve from 0.50281
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 89s 71ms/step - accuracy: 0.5594 - loss: 1.7555 - val_accuracy: 0.7101 - val_loss: 1.0222 - learning_rate: 1.0000e-05
Epoch 52/109
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.6170 - loss: 1.3811
Epoch 52: val_loss did not improve from 0.50281
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 89s 71ms/step - accuracy: 0.6171 - loss: 1.3811 - val_accuracy: 0.7413 - val_loss: 0.8308 - learning_rate: 1.0000e-05
Epoch 53/109
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.6454 - loss: 1.1707
Epoch 53: val_loss did not improve from 0.50281
1250/1250

In [3]:
# 테스트 데이터셋을 이용해 모델을 평가합니다.
model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9249 - loss: 0.2297


[0.23493364453315735, 0.9228000044822693]